In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

# import selenium

In [2]:
pd.__version__

'1.1.5'

# Scraping Job Opportunity in LinkedIn

In [ ]:
html_text = requests.get('https://www.linkedin.com/jobs/search/?currentJobId=3646641361&keywords=python&origin=JOB_COLLECTION_PAGE_SEARCH_BUTTON&refresh=true').text

soup = BeautifulSoup(html_text, 'html.parser')

jobs = soup.find_all('li', class_='''ember-view''')
for job in jobs:
    print(job)



# Scraping Job Opportunity in timesjobs.com

## Without any Filtering

In [ ]:
html_text = requests.get('https://www.timesjobs.com/candidate/job-search.html?searchType=personalizedSearch&from=submit&txtKeywords=python&txtLocation=').text

soup = BeautifulSoup(html_text, 'html.parser')

jobs = soup.find_all('li', class_='clearfix job-bx wht-shd-bx')

job_posts = pd.DataFrame()

for job in jobs:
    company = job.find('h3', class_='joblist-comp-name').text.strip().upper()
    position = job.find('a', target='_blank').text.strip()
    skills = job.find('span', class_='srp-skills').text.strip().split('  ,  ')
    posted = job.find('span', class_='sim-posted').text.strip()
    more_info = job.header.h2.a['href']
    job_post = pd.DataFrame([[company, position, skills, posted, more_info]], columns=['Company', 'Position', 'Skills', 'Posted', 'More Info'], index=[0])
    job_posts = job_posts.append(job_post, ignore_index=True)

print('This is the result of scraping TimesJobs.com')
display(job_posts)

## With Filtering

In [ ]:
html_text = requests.get('https://www.timesjobs.com/candidate/job-search.html?from=submit&luceneResultSize=25&txtKeywords=python&postWeek=60&searchType=personalizedSearch&actualTxtKeywords=python&searchBy=0&rdoOperator=OR&pDate=I&sequence=1&startPage=1').text
soup = BeautifulSoup(html_text, 'html.parser')
jobs = soup.find_all('li', class_='clearfix job-bx wht-shd-bx')
job_posts = pd.DataFrame()
for job in jobs:
    posted = job.find('span', class_='sim-posted').text.strip()
    if 'few' not in posted:
        company = job.find('h3', class_='joblist-comp-name').text.strip().upper()
        position = job.find('a', target='_blank').text.strip()
        skills = job.find('span', class_='srp-skills').text.strip().split('  ,  ')
        more_info = job.header.h2.a['href']
        job_post = pd.DataFrame([[company, position, skills, posted, more_info]], columns=['Company', 'Position', 'Skills', 'Posted', 'More Info'], index=[0])
        job_posts = job_posts.append(job_post, ignore_index=True)

print('This is the result of scraping TimesJobs.com')
display(job_posts)

## Looping through pages

In [ ]:
html_text = requests.get('https://www.timesjobs.com/candidate/job-search.html?from=submit&luceneResultSize=25&txtKeywords=python&postWeek=60&searchType=personalizedSearch&actualTxtKeywords=python&searchBy=0&rdoOperator=OR&pDate=I&sequence=1&startPage=1').text
soup = BeautifulSoup(html_text, 'html.parser')
pages = soup.find('div', {'class':'srp-pagination clearfix'})
job_posts = pd.DataFrame()
page_now = 1
for page in pages:
    html_text = requests.get(f'https://www.timesjobs.com/candidate/job-search.html?from=submit&luceneResultSize=25&txtKeywords=python&postWeek=60&searchType=personalizedSearch&actualTxtKeywords=python&searchBy=0&rdoOperator=OR&pDate=I&sequence={page_now}&startPage=1').text
    soup = BeautifulSoup(html_text, 'html.parser')
    jobs = soup.find_all('li', class_='clearfix job-bx wht-shd-bx')
    if 'Next' not in page.text.strip():
        for job in jobs:
            posted = job.find('span', class_='sim-posted').text.strip()
            company = job.find('h3', class_='joblist-comp-name').text.strip().upper()
            position = job.find('a', target='_blank').text.strip()
            skills = job.find('span', class_='srp-skills').text.strip().split('  ,  ')
            more_info = job.header.h2.a['href']
            job_post = pd.DataFrame([[company, position, skills, posted, more_info]], columns=['Company', 'Position', 'Skills', 'Posted', 'More Info'], index=[0])
            job_post['Page'] = page_now
            job_posts = job_posts.append(job_post, ignore_index=True)
    else:
        break
    page_now+=1

print('This is the result of scraping TimesJobs.com')
display(job_posts)